In [5]:
import os
import re
import json
import traceback
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

In [9]:
def criar_docx_de_json(arquivo_json, arquivo_saida='curriculo.docx'):
    """Cria um documento Word formatado a partir de dados de um currículo em JSON."""
    # Carregar dados JSON
    with open(arquivo_json, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    # Diagnóstico para exibir estrutura dos dados JSON
    print("Estrutura dos Dados JSON:")
    print(json.dumps(dados, indent=2))
    
    # Verificar se os dados são um dicionário
    if isinstance(dados, str):
        try:
            dados = json.loads(dados)
        except json.JSONDecodeError:
            print("Erro: Não foi possível interpretar a string JSON")
            return

    # Criar um novo Documento
    doc = Document()

    # Definir fonte padrão
    estilo = doc.styles['Normal']
    estilo.font.name = 'Calibri'
    estilo.font.size = Pt(11)

    # Seção de Informações Pessoais
    informacoes_pessoais = dados.get('informacoes_pessoais', {})
    if not isinstance(informacoes_pessoais, dict):
        informacoes_pessoais = {}
    
    # Nome (Cabeçalho)
    nome = informacoes_pessoais.get('nome', 'Nome Não Encontrado')
    paragrafo_nome = doc.add_paragraph(nome)
    
    if paragrafo_nome.runs:
        nome_run = paragrafo_nome.runs[0]
        nome_run.bold = True
        nome_run.font.size = Pt(16)
        nome_run.font.color.rgb = RGBColor(0, 0, 0)  # Define a cor preta
    paragrafo_nome.alinhamento = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Informações de Contato
    contato = f"""Cidade: {informacoes_pessoais.get('cidade', 'N/A')}
Bairro: {informacoes_pessoais.get('bairro', 'N/A')}
Email: {informacoes_pessoais.get('email', 'N/A')} 
Telefone: {informacoes_pessoais.get('telefone', 'N/A')}
Posição: {informacoes_pessoais.get('cargo', 'N/A')}"""
        
    paragrafo_contato = doc.add_paragraph(contato)
    paragrafo_contato.alinhamento = WD_PARAGRAPH_ALIGNMENT.LEFT

    # Resumo de Qualificações
    qualificacoes = dados.get('resumo_qualificacoes', [])
    if qualificacoes and isinstance(qualificacoes, list):
        doc.add_heading('Resumo de Qualificações', level=2)
        
        primeira_qualificacao = qualificacoes[0] if qualificacoes else {}
        if isinstance(primeira_qualificacao, dict):
            resumo = primeira_qualificacao.get('resumo', '')
            if resumo:
                doc.add_paragraph(resumo)

            qualificacoes_chave = primeira_qualificacao.get('qualificacoes_chave', [])
            if qualificacoes_chave:
                lista_qualificacoes = doc.add_paragraph()
                for qual in qualificacoes_chave:
                    texto_qualificacao = qual.get('qualificacao', '') if isinstance(qual, dict) else str(qual)
                    lista_qualificacoes.add_run('• ' + texto_qualificacao + '\n')

    # Experiência Profissional
    doc.add_heading('Experiência Profissional', level=2)
    experiencias = dados.get('experiencia_profissional', [])
    
    for trabalho in experiencias:
        if not isinstance(trabalho, dict):
            continue
        
        cabecalho_trabalho = doc.add_paragraph()
        cargo = trabalho.get('cargo', 'Cargo Não Especificado')
        empresa = trabalho.get('empresa', 'Empresa Não Especificada')
        
        cabecalho_trabalho.add_run(cargo).bold = True
        cabecalho_trabalho.add_run(' - ' + empresa).italic = True
        
        doc.add_paragraph(trabalho.get('periodo', 'Período Não Especificado'))
        
        atividades = trabalho.get('atividades', [])
        if atividades:
            lista_atividades = doc.add_paragraph()
            for atividade in atividades:
                texto_atividade = atividade.get('atividade', '') if isinstance(atividade, dict) else str(atividade)
                lista_atividades.add_run('• ' + texto_atividade + '\n')
        
        doc.add_paragraph()

    # Salvar o documento
    doc.save(arquivo_saida)
    print(f"Currículo salvo em {arquivo_saida}")

In [10]:
criar_docx_de_json('dados_curriculo_extraidos.json')

Estrutura dos Dados JSON:
{
  "informacoes_pessoais": {
    "nome": "Gustavo Lacerda",
    "cidade": "Fortaleza, Cear\u00e1, Brasil",
    "email": "",
    "telefone": "",
    "cargo": "Engenheiro Civil"
  },
  "resumo_qualificacoes": [
    {
      "resumo": "Engenheiro civil motivado, com experi\u00eancia em gerenciamento de contratos, projetos e obra, medi\u00e7\u00f5es e elabora\u00e7\u00e3o de cronogramas.",
      "qualificacoes_chave": [
        {
          "qualificacao": "Experi\u00eancia em gerenciamento de projetos e equipes"
        },
        {
          "qualificacao": "Conhecimento em m\u00e9todos construtivos e saneamento b\u00e1sico"
        }
      ]
    }
  ],
  "experiencia_profissional": [
    {
      "empresa": "MRV&CO",
      "cargo": "Auxiliar de Engenharia - Instala\u00e7\u00f5es",
      "periodo": "abril de 2024 - agosto de 2024",
      "atividades": [
        {
          "atividade": "Acompanhamento e apoio \u00e0 infraestrutura, instala\u00e7\u00f5es el\u00e9tr